In [ ]:
%run Utils

In [ ]:
replace_workspace_id = "904f9388-f876-4176-be2a-6ef7d62d6544"
replace_download_data_nbid = "d48a1f2f-8392-4fba-bd05-f76f4b978bfe"
replace_create_table_nbid = "b052405e-a729-4316-97bf-bbb339f86985"
replace_create_starschema_nbid = "63eb8687-b7ae-4d84-9c5c-2d96b82ce74b"

In [ ]:
download_data_nbid = "ac93581d-b856-44a4-9ca5-27e4fe2ced26"
create_table_nbid = "0fae1b67-623e-47f1-8a14-cf1c669bf976"
create_starschema_nbid = "bda2663f-3a0d-4e1a-b1df-bd6dcd8c9cec"

In [ ]:
config_dict = get_config_dict()

#create the sub-directory in Files folder where the CSV files will be downloaded
lakehouse_dir = "Files/cmsdemofiles/cms_pipeline.DataPipeline"
lakehouse_dir_abfss_full_path = get_full_abfss_path(config_dict['workspace_id'], config_dict['lakehouse_id'], lakehouse_dir)

#notebookutils.fs.mkdirs(lakehouse_dir_abfss_full_path)

print(f"lakehouse_dir: {lakehouse_dir}")
print(f"lakehouse_dir_abfss_full_path: {lakehouse_dir_abfss_full_path}")

In [ ]:
#local mount path is required for plain python to save files to Lakehouse file section
mount_point = "/mnt/lakehouse/" + config_dict["lakehouse_name"] + "/" + lakehouse_dir
lakehouse_dir_local_mount_path = mount_path_return_local_path(lakehouse_dir_abfss_full_path, mount_point)

print(f'mount point: {mount_point}')
print(f"lakehouse_dir_local_path: {lakehouse_dir_local_mount_path}")

In [ ]:
pipeline_json_path = lakehouse_dir_local_mount_path + "/pipeline-content.json"
print(pipeline_json_path)
platform_file_path = lakehouse_dir_local_mount_path + "/.platform"
print(platform_file_path)

In [ ]:
def get_file_contents(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        pipeline_content_payload = file.read()
    return pipeline_content_payload



In [ ]:
platform_file_payload =  get_file_contents(platform_file_path)
print(platform_file_payload)


In [ ]:
pipeline_json_payload =  get_file_contents(pipeline_json_path)
print(pipeline_json_payload)


In [ ]:
pipeline_json_payload = pipeline_json_payload.replace(replace_workspace_id, config_dict['workspace_id'])
pipeline_json_payload = pipeline_json_payload.replace(replace_download_data_nbid, download_data_nbid)
pipeline_json_payload = pipeline_json_payload.replace(replace_create_table_nbid, create_table_nbid)
pipeline_json_payload = pipeline_json_payload.replace(replace_create_starschema_nbid, create_starschema_nbid)

print(pipeline_json_payload)

In [ ]:
notebookutils.fs.unmount(mount_point)

In [ ]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import json
import fnmatch
import re
import os


# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python
# FabricRestClient uses the default credentials of the executing user
client = fabric.FabricRestClient()



In [ ]:
import base64

request_body = {
    "displayName": "cms_pipeline",
    "description": "cms_pipeline to ingest and process data",
    "definition" : {
        "parts": [
            {
                "path": "pipeline-content.json",
                "payload": base64.b64encode(pipeline_json_payload.encode('utf-8')),
                "payloadType": "InlineBase64"
            },
            {
                "path": ".platform",
                "payload": base64.b64encode(platform_file_payload.encode('utf-8')),
                "payloadType": "InlineBase64"
            }
        ]
    }
}

In [ ]:
workspace_id = config_dict['workspace_id']
create_pipeline_uri = f"v1/workspaces/{workspace_id}/dataPipelines"
response = client.request(method = "POST", path_or_url=create_pipeline_uri, lro_wait=True, json = request_body)
print(response.status_code)
print(response.text)

In [ ]:
if response.status_code in [200, 201, 202]:
    print("Pipeline created successfully.")
    print("Response content:")
    print(response.json() if response.headers.get("Content-Type", "").startswith("application/json") else response.text)
else:
    print(f"Request failed with status code {response.status_code}")
    print("Response content:")
    print(response.text)
